In [20]:
import pandas as pd
from sklearn.metrics import roc_auc_score
import torch
import torch.nn as nn
import torch.optim as optim 
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
import numpy as np
import os
from tqdm import tqdm

file = 'NT'

file_prefixes = ["smiles","DMPNN", "ecfp", "mfbert", "padel"]
file_extension = ".csv"

file_prefix = f"{file}_Feature_fusion/data_train"
file_extension = ".csv"

val_file_prefix = f"{file}_Feature_fusion/data_valid"
val_file_extension = ".csv"

start_index = 0
end_index = 4

auc_scores = []
acc_scores = []
f1_scores = []
recall_scores = []
mcc_scores = []


for i in tqdm(range(start_index, end_index + 1), desc="Processing files"):  # 使用 tqdm 添加进度条  
    combined_data_train = None
    combined_data_valid = None
    combined_data_test = None
    for file_feature in file_prefixes:
        #从randomi/中获取train，valid，test    
        
        file_path_train = file + "/" +'random' + str(i) + "/" + file_feature + "/" + "train" + file_extension
        data_train = pd.read_csv(file_path_train) 
                
        file_path_valid = file + "/" +'random' + str(i) + "/" + file_feature + "/" + "valid" + file_extension
        data_valid = pd.read_csv(file_path_valid) 

        file_path_test = file + "/" +'random' + str(i) + "/" + file_feature + "/" + "test" + file_extension
        data_test = pd.read_csv(file_path_test)   
        
        data_train = pd.read_csv(file_path_train)     
        data_valid = pd.read_csv(file_path_valid)   
        data_test  = pd.read_csv(file_path_test) 
        
        if combined_data_train is None:
            combined_data_train = data_train
        if combined_data_valid is None:
            combined_data_valid = data_valid
        if combined_data_test is None:   
            combined_data_test  = data_test
        else:
            combined_data_train = pd.concat([combined_data_train, data_train], axis=1)     
            combined_data_valid = pd.concat([combined_data_valid, data_valid], axis=1) 
            combined_data_test = pd.concat([combined_data_test, data_test], axis=1) 
    
    folder_path = f'{file}_Feature_fusion'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    combined_data_train.to_csv(f'{file}_Feature_fusion/data_train{i}.csv', index=False)
    combined_data_valid.to_csv(f'{file}_Feature_fusion/data_valid{i}.csv', index=False)
    combined_data_test.to_csv(f'{file}_Feature_fusion/data_test{i}.csv', index=False)


Processing files: 100%|██████████| 5/5 [00:18<00:00,  3.70s/it]


In [21]:
file_prefix = f"{file}_Feature_fusion/data_train"
file_extension = ".csv"

val_file_prefix = f"{file}_Feature_fusion/data_valid"
val_file_extension = ".csv"

for i in range(start_index, end_index+1):
    
    # 读取数据
    file_path = file_prefix + str(i) + file_extension
    data = pd.read_csv(file_path)
    null_indices = None  # 初始化 null_indices 为 None
    if data.isnull().values.any():  # 判断是否有空值
        print("数据中存在空值，空值坐标为：")
        null_indices = np.where(data.isnull())  # 获取空值的坐标
        if null_indices is not None:  # 再次判断 null_indices 是否有值
            for row_index, col_index in zip(null_indices[0], null_indices[1]):
                print(f"行 {row_index}，列 {col_index}")
    else:
        print("数据中没有空值")
    print(f"该文件的行数为：{data.shape[0]}，列数为：{data.shape[1]}")
    

数据中没有空值
该文件的行数为：2087，列数为：3538
数据中没有空值
该文件的行数为：2087，列数为：3538
数据中没有空值
该文件的行数为：2087，列数为：3538
数据中没有空值
该文件的行数为：2087，列数为：3538
数据中没有空值
该文件的行数为：2087，列数为：3538


In [26]:
# 读取CSV文件
for i in range(start_index, end_index+1):
    
    # 读取数据
    file_path = file_prefix + str(i) + file_extension
    data = pd.read_csv(file_path)
    
    first_row = data.iloc[0]
    print(first_row)
    
    # 提取特征和标签
    x_input = data.drop(['SMILES'], axis=1).values
    y_output = data['active'].values
    
        # 遍历数据，将 inf 赋值为 0
    x_input = np.nan_to_num(x_input, posinf=0, neginf=0)
    
#        # 检查并输出非 float64 数据的坐标
#    for row_index, row in enumerate(x_input):
#        for col_index, value in enumerate(row):
#            if x_input.dtype!= np.float64:
#                print(f"非 float64 数据位于 ({row_index}, {col_index})，值为: {value}")
#    
    #print('x_input',x_input)
    #print('y_output',y_output)
    
    
    val_x_input = data.drop(['SMILES'], axis=1).values
    val_y_output = data['active'].values
    
    val_x_input = np.nan_to_num(x_input, posinf=0, neginf=0)
    
#    # 查找 x_input 中的无穷值位置
#    infinity_positions_x = np.argwhere(np.isinf(x_input))
#    if len(infinity_positions_x) > 0:
#        column_names = data.drop(['SMILES'], axis=1).columns
#        infinity_column_names_x = [column_names[pos[1]] for pos in infinity_positions_x]
#        print("在 x_input 中的无穷值所在列名：", infinity_column_names_x, '         位置：',infinity_positions_x)
#    else:
#        print("在 x_input 中没有无穷值")
#
#    # 查找 val_x_input 中的无穷值位置
#    infinity_positions_val_x = np.argwhere(np.isinf(val_x_input))
#    if len(infinity_positions_val_x) > 0:
#        column_names = data.drop(['SMILES'], axis=1).columns
#        infinity_column_names_val_x = [column_names[pos[1]] for pos in infinity_positions_val_x]
#        print("在 val_x_input 中的无穷值所在列名：", infinity_column_names_val_x, '        位置：',infinity_positions_x)
#    else:
#        print("在 val_x_input 中没有无穷值")
    
        # 标准化特征（假设特征是数值型的）
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    x_input = scaler.fit_transform(x_input)
    val_x_input = scaler.fit_transform(val_x_input)

    # 数据放缩处理（例如，将特征值缩放到 0 到 1 之间）
    from sklearn.preprocessing import MinMaxScaler
    min_max_scaler = MinMaxScaler(feature_range=(0, 1))
    x_input = min_max_scaler.fit_transform(x_input)
    val_x_input = min_max_scaler.fit_transform(val_x_input)
    
        # 转换为Tensor
    x = torch.Tensor(x_input)
    y_true = torch.Tensor(y_output).view(-1, 1)    
    
    val_x = torch.Tensor(val_x_input)
    val_y_true = torch.Tensor(val_y_output).view(-1, 1)
    
    
    # 数据维度
    input_dim = x.shape[1]
    output_dim = 1
    hidden_dim1 = 256
    hidden_dim2 = 128
    hidden_dim3 = 32
    learning_rate = 0.001
    num_epochs = 100
    weight_decay = 0.001
    # MLP模型定义
    mlp = nn.Sequential(  
        nn.Linear(input_dim, hidden_dim1),  # 第一层隐藏层  
        nn.ReLU(),  
        nn.Linear(hidden_dim1, hidden_dim2),  # 第二层隐藏层  
        nn.ReLU(),  
        nn.Linear(hidden_dim2, hidden_dim3),  # 第三层隐藏层  
        nn.ReLU(),  
        nn.Linear(hidden_dim3, output_dim),  # 输出层  
        nn.Sigmoid()  
    )  
        
    # 优化器和损失函数
    optimizer = torch.optim.Adam(mlp.parameters(), lr=learning_rate,weight_decay=weight_decay)
    #loss_func =  nn.BCEWithLogitsLoss()
    loss_func =  nn.MSELoss()
    best_val_loss = float('inf')
    num_iterations_without_improvement = 0  # Initialize counter for iterations without improvement. 
    
    for epoch in range(num_epochs):
        
        print(f"开始训练第{epoch}组epochs")
        prediction = mlp(x)
        #print('prediction',prediction)
        
        #prediction_np = prediction.detach().numpy()
        #df = pd.DataFrame(prediction_np)
        #df.to_csv('prediction.csv', index=False)
        
        loss = loss_func(prediction, y_true)
        print('loss',loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print(optimizer)
        
        with torch.no_grad():
            val_prediction=mlp(val_x)
            val_loss = loss_func(val_prediction, val_y_true)
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {loss.item()}, Val Loss: {val_loss.item()}")
        
        #print("检查数据和模型输出:")
        #print("是否存在 NaN 或无穷大值在训练预测结果中:", torch.isnan(prediction).any() or torch.isinf(prediction).any())
        #print("是否存在 NaN 或无穷大值在验证预测结果中:", torch.isnan(val_prediction).any() or torch.isinf(val_prediction).any())
        #print("是否存在 NaN 或无穷大值在训练真实值中:", torch.isnan(y_true).any() or torch.isinf(y_true).any())
        #print("是否存在 NaN 或无穷大值在验证真实值中:", torch.isnan(val_y_true).any() or torch.isinf(val_y_true).any())
        #
          
        if val_loss < best_val_loss:
            best_val_loss = val_loss  
            num_iterations_without_improvement = 0
            torch.save(mlp.state_dict(), f"models_{file}/model_{i+1}.pt")
            #torch.save(mlp, "model_saved/mlp1.pt")
            print(f'model_{i}训练结果更新为第{epoch}个模型')
            best_epoch = epoch
        else :
            num_iterations_without_improvement += 1  
            if num_iterations_without_improvement == 300:
                print('300次迭代没有更新，结束迭代')
                print(f'最棒的模型是第{best_epoch}个epoch')
                break
        print('         ')
        

    prediction_np = prediction.data.numpy()
        # 将 prediction 添加到 DataFrame
    #data['prediction'] = prediction_np
        # 将 DataFrame 保存到 CSV 文件
    #data.to_csv('output/output_train.csv', index=True)
    
    # 预测并计算AUC
    prediction_np = prediction.detach().numpy().flatten()
    auc = roc_auc_score(y_true, prediction_np)
    
    threshold = 0.5
    binary_prediction = np.where(prediction_np > threshold, 1, 0)
    
    acc = accuracy_score(y_true, binary_prediction)
    f1 = f1_score(y_true, binary_prediction)
    recall = recall_score(y_true, binary_prediction)
    mcc = matthews_corrcoef(y_true, binary_prediction)
    output_file = "train_scores.csv"
    
    
    auc_scores.append(auc)
    acc_scores.append(acc)
    f1_scores.append(f1)
    recall_scores.append(recall)
    mcc_scores.append(mcc)
    
    print('第',i,'组模型')
    print("AUC:", auc)
    print("ACC:", acc)
    print("F1:", f1)
    print("Recall:", recall)
    print("MCC:", mcc)
    print('    ')
      
    

SMILES    OC(CNCC1=CC=CC=C1)COCC2=CC=CC=C2
active                                   1
0                                      0.0
1                                      0.0
2                                 0.088121
                        ...               
1439                              0.094399
1440                              0.028085
1441                              0.162963
1442                              0.610029
1443                              0.235897
Name: 0, Length: 3538, dtype: object
开始训练第0组epochs
loss tensor(0.2595, grad_fn=<MseLossBackward>)
Epoch 1/100, Train Loss: 0.2594864070415497, Val Loss: 0.2422243058681488
model_0训练结果更新为第0个模型
         
开始训练第1组epochs
loss tensor(0.2422, grad_fn=<MseLossBackward>)
Epoch 2/100, Train Loss: 0.2422243058681488, Val Loss: 0.22606632113456726
model_0训练结果更新为第1个模型
         
开始训练第2组epochs
loss tensor(0.2261, grad_fn=<MseLossBackward>)
Epoch 3/100, Train Loss: 0.22606632113456726, Val Loss: 0.21942368149757385
model_0训练结果更新为第2个模型
  

Epoch 49/100, Train Loss: 0.05965893343091011, Val Loss: 0.06173619627952576
         
开始训练第49组epochs
loss tensor(0.0617, grad_fn=<MseLossBackward>)
Epoch 50/100, Train Loss: 0.06173619627952576, Val Loss: 0.05785660445690155
model_0训练结果更新为第49个模型
         
开始训练第50组epochs
loss tensor(0.0579, grad_fn=<MseLossBackward>)
Epoch 51/100, Train Loss: 0.05785660445690155, Val Loss: 0.058149319142103195
         
开始训练第51组epochs
loss tensor(0.0581, grad_fn=<MseLossBackward>)
Epoch 52/100, Train Loss: 0.058149319142103195, Val Loss: 0.05829082429409027
         
开始训练第52组epochs
loss tensor(0.0583, grad_fn=<MseLossBackward>)
Epoch 53/100, Train Loss: 0.05829082429409027, Val Loss: 0.05541408434510231
model_0训练结果更新为第52个模型
         
开始训练第53组epochs
loss tensor(0.0554, grad_fn=<MseLossBackward>)
Epoch 54/100, Train Loss: 0.05541408434510231, Val Loss: 0.05663150176405907
         
开始训练第54组epochs
loss tensor(0.0566, grad_fn=<MseLossBackward>)
Epoch 55/100, Train Loss: 0.05663150176405907, Val Loss: 0.055

SMILES    NCCS(O)(=O)=O
active                1
0                   0.0
1              0.008748
2              0.074378
              ...      
1439           0.074249
1440           0.001213
1441           0.028169
1442           0.524964
1443           0.071795
Name: 0, Length: 3538, dtype: object
开始训练第0组epochs
loss tensor(0.2369, grad_fn=<MseLossBackward>)
Epoch 1/100, Train Loss: 0.23686976730823517, Val Loss: 0.22683587670326233
model_1训练结果更新为第0个模型
         
开始训练第1组epochs
loss tensor(0.2268, grad_fn=<MseLossBackward>)
Epoch 2/100, Train Loss: 0.22683587670326233, Val Loss: 0.21971289813518524
model_1训练结果更新为第1个模型
         
开始训练第2组epochs
loss tensor(0.2197, grad_fn=<MseLossBackward>)
Epoch 3/100, Train Loss: 0.21971289813518524, Val Loss: 0.22126643359661102
         
开始训练第3组epochs
loss tensor(0.2213, grad_fn=<MseLossBackward>)
Epoch 4/100, Train Loss: 0.22126643359661102, Val Loss: 0.22014713287353516
         
开始训练第4组epochs
loss tensor(0.2201, grad_fn=<MseLossBackward>)
Epoch 5/10

model_1训练结果更新为第48个模型
         
开始训练第49组epochs
loss tensor(0.1127, grad_fn=<MseLossBackward>)
Epoch 50/100, Train Loss: 0.1126563772559166, Val Loss: 0.11372201889753342
         
开始训练第50组epochs
loss tensor(0.1137, grad_fn=<MseLossBackward>)
Epoch 51/100, Train Loss: 0.11372201889753342, Val Loss: 0.11062700301408768
model_1训练结果更新为第50个模型
         
开始训练第51组epochs
loss tensor(0.1106, grad_fn=<MseLossBackward>)
Epoch 52/100, Train Loss: 0.11062700301408768, Val Loss: 0.1078716367483139
model_1训练结果更新为第51个模型
         
开始训练第52组epochs
loss tensor(0.1079, grad_fn=<MseLossBackward>)
Epoch 53/100, Train Loss: 0.1078716367483139, Val Loss: 0.10907694697380066
         
开始训练第53组epochs
loss tensor(0.1091, grad_fn=<MseLossBackward>)
Epoch 54/100, Train Loss: 0.10907694697380066, Val Loss: 0.10843252390623093
         
开始训练第54组epochs
loss tensor(0.1084, grad_fn=<MseLossBackward>)
Epoch 55/100, Train Loss: 0.10843252390623093, Val Loss: 0.10326030105352402
model_1训练结果更新为第54个模型
         
开始训练第55组epochs


开始训练第0组epochs
loss tensor(0.2442, grad_fn=<MseLossBackward>)
Epoch 1/100, Train Loss: 0.2442186176776886, Val Loss: 0.231735497713089
model_2训练结果更新为第0个模型
         
开始训练第1组epochs
loss tensor(0.2317, grad_fn=<MseLossBackward>)
Epoch 2/100, Train Loss: 0.231735497713089, Val Loss: 0.22119605541229248
model_2训练结果更新为第1个模型
         
开始训练第2组epochs
loss tensor(0.2212, grad_fn=<MseLossBackward>)
Epoch 3/100, Train Loss: 0.22119605541229248, Val Loss: 0.21745924651622772
model_2训练结果更新为第2个模型
         
开始训练第3组epochs
loss tensor(0.2175, grad_fn=<MseLossBackward>)
Epoch 4/100, Train Loss: 0.21745924651622772, Val Loss: 0.21670976281166077
model_2训练结果更新为第3个模型
         
开始训练第4组epochs
loss tensor(0.2167, grad_fn=<MseLossBackward>)
Epoch 5/100, Train Loss: 0.21670976281166077, Val Loss: 0.21118268370628357
model_2训练结果更新为第4个模型
         
开始训练第5组epochs
loss tensor(0.2112, grad_fn=<MseLossBackward>)
Epoch 6/100, Train Loss: 0.21118268370628357, Val Loss: 0.20410610735416412
model_2训练结果更新为第5个模型
         
开始训

Epoch 49/100, Train Loss: 0.005198247265070677, Val Loss: 0.005264970473945141
         
开始训练第49组epochs
loss tensor(0.0053, grad_fn=<MseLossBackward>)
Epoch 50/100, Train Loss: 0.005264970473945141, Val Loss: 0.005803239531815052
         
开始训练第50组epochs
loss tensor(0.0058, grad_fn=<MseLossBackward>)
Epoch 51/100, Train Loss: 0.005803239531815052, Val Loss: 0.007030915468931198
         
开始训练第51组epochs
loss tensor(0.0070, grad_fn=<MseLossBackward>)
Epoch 52/100, Train Loss: 0.007030915468931198, Val Loss: 0.009289269335567951
         
开始训练第52组epochs
loss tensor(0.0093, grad_fn=<MseLossBackward>)
Epoch 53/100, Train Loss: 0.009289269335567951, Val Loss: 0.006910574622452259
         
开始训练第53组epochs
loss tensor(0.0069, grad_fn=<MseLossBackward>)
Epoch 54/100, Train Loss: 0.006910574622452259, Val Loss: 0.004410160705447197
model_2训练结果更新为第53个模型
         
开始训练第54组epochs
loss tensor(0.0044, grad_fn=<MseLossBackward>)
Epoch 55/100, Train Loss: 0.004410160705447197, Val Loss: 0.0064305122941

SMILES    C1=C(C2=C(N=C1N3CCN(CC)CC3)CCCCCC2)C4=CC=C(F)C=C4
active                                                    1
0                                                   0.15136
1                                                       0.0
2                                                  0.089399
                                ...                        
1439                                                0.31617
1440                                               0.047363
1441                                               0.323944
1442                                               0.627516
1443                                               0.364103
Name: 0, Length: 3538, dtype: object
开始训练第0组epochs
loss tensor(0.2558, grad_fn=<MseLossBackward>)
Epoch 1/100, Train Loss: 0.2558170557022095, Val Loss: 0.2452452927827835
model_3训练结果更新为第0个模型
         
开始训练第1组epochs
loss tensor(0.2452, grad_fn=<MseLossBackward>)
Epoch 2/100, Train Loss: 0.2452452927827835, Val Loss: 0.22780857980251312
mo

model_3训练结果更新为第47个模型
         
开始训练第48组epochs
loss tensor(0.1064, grad_fn=<MseLossBackward>)
Epoch 49/100, Train Loss: 0.10635360330343246, Val Loss: 0.10717234760522842
         
开始训练第49组epochs
loss tensor(0.1072, grad_fn=<MseLossBackward>)
Epoch 50/100, Train Loss: 0.10717234760522842, Val Loss: 0.10874257981777191
         
开始训练第50组epochs
loss tensor(0.1087, grad_fn=<MseLossBackward>)
Epoch 51/100, Train Loss: 0.10874257981777191, Val Loss: 0.10373528301715851
model_3训练结果更新为第50个模型
         
开始训练第51组epochs
loss tensor(0.1037, grad_fn=<MseLossBackward>)
Epoch 52/100, Train Loss: 0.10373528301715851, Val Loss: 0.09944802522659302
model_3训练结果更新为第51个模型
         
开始训练第52组epochs
loss tensor(0.0994, grad_fn=<MseLossBackward>)
Epoch 53/100, Train Loss: 0.09944802522659302, Val Loss: 0.0985875129699707
model_3训练结果更新为第52个模型
         
开始训练第53组epochs
loss tensor(0.0986, grad_fn=<MseLossBackward>)
Epoch 54/100, Train Loss: 0.0985875129699707, Val Loss: 0.10042066872119904
         
开始训练第54组epochs

SMILES    CC(=O)CC(C)C
active               1
0                  0.0
1             0.008262
2             0.006112
              ...     
1439               0.0
1440          0.001266
1441          0.028571
1442          0.438565
1443          0.066667
Name: 0, Length: 3538, dtype: object
开始训练第0组epochs
loss tensor(0.2386, grad_fn=<MseLossBackward>)
Epoch 1/100, Train Loss: 0.2385978102684021, Val Loss: 0.22959575057029724
model_4训练结果更新为第0个模型
         
开始训练第1组epochs
loss tensor(0.2296, grad_fn=<MseLossBackward>)
Epoch 2/100, Train Loss: 0.22959575057029724, Val Loss: 0.22051414847373962
model_4训练结果更新为第1个模型
         
开始训练第2组epochs
loss tensor(0.2205, grad_fn=<MseLossBackward>)
Epoch 3/100, Train Loss: 0.22051414847373962, Val Loss: 0.2199510782957077
model_4训练结果更新为第2个模型
         
开始训练第3组epochs
loss tensor(0.2200, grad_fn=<MseLossBackward>)
Epoch 4/100, Train Loss: 0.2199510782957077, Val Loss: 0.22099708020687103
         
开始训练第4组epochs
loss tensor(0.2210, grad_fn=<MseLossBackward>)
Epoc

Epoch 53/100, Train Loss: 0.10645665973424911, Val Loss: 0.13498011231422424
         
开始训练第53组epochs
loss tensor(0.1350, grad_fn=<MseLossBackward>)
Epoch 54/100, Train Loss: 0.13498011231422424, Val Loss: 0.10917175561189651
         
开始训练第54组epochs
loss tensor(0.1092, grad_fn=<MseLossBackward>)
Epoch 55/100, Train Loss: 0.10917175561189651, Val Loss: 0.11607548594474792
         
开始训练第55组epochs
loss tensor(0.1161, grad_fn=<MseLossBackward>)
Epoch 56/100, Train Loss: 0.11607548594474792, Val Loss: 0.10641437768936157
model_4训练结果更新为第55个模型
         
开始训练第56组epochs
loss tensor(0.1064, grad_fn=<MseLossBackward>)
Epoch 57/100, Train Loss: 0.10641437768936157, Val Loss: 0.10172152519226074
model_4训练结果更新为第56个模型
         
开始训练第57组epochs
loss tensor(0.1017, grad_fn=<MseLossBackward>)
Epoch 58/100, Train Loss: 0.10172152519226074, Val Loss: 0.1082935556769371
         
开始训练第58组epochs
loss tensor(0.1083, grad_fn=<MseLossBackward>)
Epoch 59/100, Train Loss: 0.1082935556769371, Val Loss: 0.0963105

In [27]:
te_file_prefix = f"{file}_Feature_fusion/data_test"
te_file_extension = ".csv"

import os 

avg_auc = 0.0  
avg_acc = 0.0  
avg_f1 = 0.0  
avg_recall = 0.0  
avg_mcc = 0.0  


#te_file_paths = [te_file_prefix + str(i) + te_file_extension for i in range(start_index, end_index+1)]  
results = pd.DataFrame(columns=['AUC', 'Accuracy', 'F1', 'Recall', 'MCC'])   
    
#for i,te_file_path in enumerate(te_file_paths):  
for i in range(start_index, end_index): 
    te_file_path = te_file_prefix + str(i) + te_file_extension
    data_test = pd.read_csv(te_file_path)  
    if file == 'BBB':
        data_test = data_test.head(129)  

    #print(data_test)
      
    te_x_input = data_test.drop(['SMILES'], axis=1).values  
    #print('1te_x_input.shape',te_x_input.shape)
    
    te_y_true = data_test['active'].values  
    te_x_input = te_x_input.astype(np.float64)
    #print('2te_x_input.shape',te_x_input.shape)
    
    te_x_input = np.nan_to_num(te_x_input, posinf=0, neginf=0)
    
    
    print(np.isnan(te_y_true).any())  # 检查 te_y_true 是否包含 NaN
    print(np.isinf(te_y_true).any())  # 检查 te_y_true 是否包含无穷值
    
    #print('3te_x_input',te_x_input.shape)    
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    te_x_input = scaler.fit_transform(te_x_input)

    # 数据放缩处理（例如，将特征值缩放到 0 到 1 之间）
    from sklearn.preprocessing import MinMaxScaler
    min_max_scaler = MinMaxScaler(feature_range=(0, 1))
    te_x_input = min_max_scaler.fit_transform(te_x_input)
    te_x_input = torch.Tensor(te_x_input)
    #print('4te_x_input.shape',te_x_input.shape)
    mlp.load_state_dict(torch.load(f"models_{file}/model_{i+1}.pt")) 
    
    #print('39 te_x_input te_x_input ',te_x_input)
    
    #print("Length of te_prediction:", len(te_prediction))
    #print("Length of data_test:", len(data_test))
    #print("Length of data index:", len(data.index))
    #
    with torch.no_grad():    
        
       # print("47  te_x_input  te_x_input",te_x_input)
        
        te_prediction = mlp(te_x_input)  
        
        ############################################################
        te_prediction = te_prediction.numpy()
        data = pd.DataFrame(te_prediction)
        data.to_csv(f'output1/test_prediction{i+1}.csv', mode='a', index=False)
        ############################################################
        
        #print('te_y_true',len(te_y_true))
        #print('te_prediction',len(te_prediction))
        #print('te_y_true',te_y_true)
        #print('te_prediction',te_prediction)
        #print('检查 te_y_true 是否包含 NaN',np.isnan(te_y_true).any())  # 检查 te_y_true 是否包含 NaN
        #print('检查 te_y_true 是否包含无穷值',np.isinf(te_y_true).any())  # 检查 te_y_true 是否包含无穷值
        #print('检查 te_prediction 是否包含 NaN',np.isnan(te_prediction).any())  # 检查 te_prediction 是否包含 NaN
        #print('检查 te_prediction 是否包含无穷值',np.isinf(te_prediction).any())  # 检查 te_prediction 是否包含无穷值
        #
        auc = roc_auc_score(te_y_true, te_prediction)
        #print(auc)
        
        #print('54  te_predictionte_prediction',te_prediction)    
        #te_prediction = te_prediction.numpy()      
        data_test[f'te_prediction_{i+1}'] = te_prediction 
        
        auc = roc_auc_score(te_y_true, te_prediction)
        #print(auc)
        
        #data_test.to_csv(f'output1/te_prediction{i+1}.csv', mode='a', index=True)
        #print('te_prediction  ',te_prediction)
        
        auc = roc_auc_score(te_y_true, te_prediction)  
        print(f'AUC_{i+1}: {auc}')
        
        te_prediction_binary = np.where(te_prediction > 0.5, 1, 0)  
        acc = accuracy_score(te_y_true, te_prediction_binary)  
        print(f'Accuracy_{i+1}: {acc}')
        f1 = f1_score(te_y_true, te_prediction_binary)
        recall = recall_score(te_y_true, te_prediction_binary)
        mcc = matthews_corrcoef(te_y_true, te_prediction_binary)
        print(f'f1_{i+1}: {f1}')
        print(f'recall_{i+1}: {recall}')
        print(f'mcc_{i+1}: {mcc}')
        print('             ')
        print('             ')
        
        avg_auc += auc  
        avg_acc += acc  
        avg_f1 += f1  
        avg_recall += recall  
        avg_mcc += mcc
        
        results = results.append({'AUC': auc, 'Accuracy': acc, 'F1': f1, 'Recall': recall, 'MCC': mcc}, ignore_index=True) 
results.to_csv('测试集预测_results.csv', index=False)

avg_auc /= len(range(start_index, end_index)) 
avg_acc /= len(range(start_index, end_index)) 
avg_f1 /= len(range(start_index, end_index))  
avg_recall /= len(range(start_index, end_index))
avg_mcc /= len(range(start_index, end_index))

auc_te_values = results['AUC'].values
accuracy_te_values = results['Accuracy'].values
f1_te_values = results['F1'].values
recall_te_values = results['Recall'].values
mcc_te_values = results['MCC'].values

std_auc_te = np.std(auc_te_values)
std_acc_te = np.std(accuracy_te_values)
std_f1_te = np.std(f1_te_values)
std_recall_te = np.std(recall_te_values)
std_mcc_te = np.std(mcc_te_values)

print(f'Average AUC: {avg_auc:.6f}±{std_auc_te:.6f}')
print(f'Average Accuracy: {avg_acc:.6f}±{std_acc_te:.6f}')
print(f'Average F1: {avg_f1:.6f}±{std_f1_te:.6f}')
print(f'Average Recall: {avg_recall:.6f}±{std_recall_te:.6f}')
print(f'Average MCC: {avg_mcc:.6f}±{std_mcc_te:.6f}')

False
False
AUC_1: 0.8424947145877377
Accuracy_1: 0.7633587786259542
f1_1: 0.8132530120481928
recall_1: 0.7670454545454546
mcc_1: 0.5002363589372892
             
             
False
False
AUC_2: 0.895066735491483
Accuracy_2: 0.7900763358778626
f1_2: 0.8405797101449276
recall_2: 0.7591623036649214
mcc_2: 0.5710401914343242
             
             
False
False
AUC_3: 0.7687041219649915
Accuracy_3: 0.7279693486590039
f1_3: 0.7977207977207976
recall_3: 0.7608695652173914
mcc_3: 0.3897168370610816
             
             
False
False
AUC_4: 0.8927648578811369
Accuracy_4: 0.816793893129771
f1_4: 0.8545454545454545
recall_4: 0.8197674418604651
mcc_4: 0.6123306925907216
             
             
Average AUC: 0.849758±0.051296
Average Accuracy: 0.774550±0.032866
Average F1: 0.826525±0.022296
Average Recall: 0.776711±0.025031
Average MCC: 0.518331±0.084385


In [28]:
learning_rate = 0.1
num_epochs = 100
weight_decay = 0.01




learning_rate = 0.01
num_epochs = 100
weight_decay = 0.01

Average AUC: 0.943448±0.020126
Average Accuracy: 0.796512±0.098723
Average F1: 0.831558±0.094451
Average Recall: 0.774691±0.173856
Average MCC: 0.624705±0.130822





    output_dim = 1
    hidden_dim1 = 512
    hidden_dim2 = 256
    hidden_dim3 = 64
    learning_rate = 0.001
    num_epochs = 100
    weight_decay = 0.01
    
Average AUC: 0.725601±0.064932
Average Accuracy: 0.598296±0.056470
Average F1: 0.666462±0.026149
Average Recall: 0.785408±0.030126
Average MCC: 0.353964±0.151104

SyntaxError: invalid syntax (830061736.py, line 12)